In [1]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import brute
import requests
from binance.client import Client

C:\Users\Asus\anaconda3\envs\aaaa\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
api_key = 'dNC0absMhm1yaOQ5dlG9xQabyqIzpNdJo9PJ6l7snb7O17DVWWPtHvQ5GUpgm0Tq'
api_secret = 'Xs5Lo26gAAJrAN6Lg4FS9KBkJvKl7zjMP5JRxwTs9IKt1drdLopug6TS8h1asuc9'
exchange = ccxt.binance()
binance_client = Client(api_key, api_secret)

In [3]:
CANDLE_DURATION_IN_MIN = 5

# Configuración de símbolos
CCXT_TICKER_NAME = 'BTC/USDT'
TRADING_TICKER_NAME = 'btcusdt'

# Parámetros de la estrategia
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70

BB_PERIOD = 20
BB_STD_DEV = 2

# Parámetros de optimización
OPTIMIZATION_PERIODS = (10, 100, 10)

# Parámetros de gestión de riesgos
MAX_RISK_PERCENTAGE = 0.02

In [4]:
def fetch_data(ticker):
    global exchange
    bars, ticker_df = None, None

    try:
        bars = exchange.fetch_ohlcv(ticker, timeframe=f'{CANDLE_DURATION_IN_MIN}m', limit=100)
    except:
        print(f"Error in fetching data from the exchange:{ticker}")

    if bars is not None:
        ticker_df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ticker_df['Date'] = pd.to_datetime(ticker_df['timestamp'], unit='ms')
        ticker_df['symbol'] = ticker

    return ticker_df

In [5]:
def get_hash_ribbon_data():
    url = "https://api.blockchain.info/charts/hash-rate?timespan=60days&format=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["values"]
        last_30d_hash_rate = sum([item["y"] for item in data[-30:]])/30
        last_60d_hash_rate = sum([item["y"] for item in data[-60:]])/60
        return last_30d_hash_rate, last_60d_hash_rate
    else:
        print(f"Error fetching Hash Ribbon data: {response.status_code}")

In [6]:
def calculate_volume(ticker_df):
    volume = ticker_df['volume'].rolling(window=20).mean()
    return volume

In [7]:
def get_utxo_metrics(df):
    df['range'] = df['high'] - df['low']
    df['amplitude'] = df['high'] - df['low'].shift(1)
    df['volume_flow'] = df['volume'] * (df['close'] - df['open'])
    df['volume_rolling_sum'] = df['volume_flow'].rolling(window=10).sum()
    df['vortex_pos'] = abs(df['high'] - df['low'].shift(1)) / df['range'].shift(1)
    df['vortex_neg'] = abs(df['low'] - df['high'].shift(1)) / df['range'].shift(1)
    return df


In [8]:
def get_dynamic_range_nvt_signal(ticker_df):
    circulating_supply = 1000000 # replace this with the actual circulating supply value ARREGLAR
    nvt_data = ticker_df['close'] * ticker_df['volume'] / circulating_supply 
    nvt_signal = nvt_data.rolling(window=24).mean() / nvt_data.rolling(window=144).mean() 
    return nvt_signal.iloc[-1]

In [9]:
def get_metcalfe_law(ticker_df):
    active_addresses = ticker_df['volume'] / ticker_df['close']  # Supuesto: active_addresses está relacionado con el volumen y el precio de cierre
    metcalfe_law = active_addresses ** 2 / ticker_df['close']  # Supuesto: la circulación se relaciona con el precio de cierre
    return metcalfe_law.iloc[-1]

In [10]:
def get_btc_energy_value_oscillator(ticker_df):
    last_30d_hash_rate, last_60d_hash_rate = get_hash_ribbon_data()
    energy_value = ticker_df['close'] * ticker_df['volume'] / (last_30d_hash_rate * 10**12)
    oscillator = energy_value.rolling(window=28).mean() / energy_value.rolling(window=180).mean() - 1
    return oscillator.iloc[-1]

In [11]:
def get_puell_multiple(ticker_df):
    btc_mined = ticker_df['volume'].rolling(window=144).sum()
    btc_price = ticker_df['close']
    puell_multiple = btc_price / (btc_mined / 144 * ticker_df['close'].rolling(window=144).mean())
    return puell_multiple.iloc[-1]

In [12]:
weights = {
    'RSI': 0.1,
    'MACD': 0.15,
    'Bollinger Bands': 0.1,
    'Hash Ribbon': 0.1,
    'Volume': 0.1,
    'UTXO Metrics': 0.1,
    'BTC Energy Value Oscillator': 0.1,
    "Metcalfe's Law": 0.1,
    'Puell Multiple': 0.15
}

threshold = 0.1

In [13]:
df = fetch_data(CCXT_TICKER_NAME)
price_range = df['high'] - df['low']
energy_value = df['volume'] * (price_range / df['close'])
energy_value_rolling_mean = energy_value.rolling(window=24).mean()
btc_ev_oscillator = (energy_value - energy_value_rolling_mean) / energy_value_rolling_mean
BTC_EV_OSCILLATOR_BUY_THRESHOLD = btc_ev_oscillator.quantile(0.1)
BTC_EV_OSCILLATOR_BUY_THRESHOLD

-0.8319544395574054

In [14]:
# calcular la media y desviación estándar del precio de cierre
close_prices = df['close']
close_mean = close_prices.mean()
close_std = close_prices.std()

# calcular el valor de METCALFE_MULTIPLIER
METCALFE_MULTIPLIER = close_mean / close_std
METCALFE_MULTIPLIER

541.8786877744725

In [15]:
def get_trade_recommendation(ticker_df, fast_period, slow_period, weights, threshold):
    close_prices = ticker_df['close']

    # RSI
    delta = close_prices.diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=RSI_PERIOD).mean()
    avg_loss = loss.rolling(window=RSI_PERIOD).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    rsi_weight = weights['RSI']

    # MACD
    ema_fast = close_prices.ewm(span=fast_period).mean()
    ema_slow = close_prices.ewm(span=slow_period).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=9).mean()
    macd_weight = weights['MACD']

    # Bollinger Bands
    bb_middle = close_prices.rolling(window=BB_PERIOD).mean()
    bb_std = close_prices.rolling(window=BB_PERIOD).std()
    bb_upper = bb_middle + (BB_STD_DEV * bb_std)
    bb_lower = bb_middle - (BB_STD_DEV * bb_std)
    bb_weight = weights['Bollinger Bands']

    # Hash Ribbon indicator
    hashrate_30d_ma, hashrate_60d_ma = get_hash_ribbon_data()
    if hashrate_30d_ma is None or hashrate_60d_ma is None:
        print("Error fetching Hash Ribbon data, skipping this indicator")
        hash_ribbon_signal = 'WAIT'
        hash_ribbon_weight = 0
    else:
        if hashrate_30d_ma > hashrate_60d_ma:
            hash_ribbon_signal = 'BUY'
        else:
            hash_ribbon_signal = 'SELL'
        hash_ribbon_weight = weights['Hash Ribbon']

    # Volume indicator
    volume = calculate_volume(ticker_df)
    if volume.iloc[-1] > volume.iloc[-2]:
        volume_signal = 'BUY'
    else:
        volume_signal = 'SELL'
    volume_weight = weights['Volume']

    # UTXO metrics indicator
     # Add UTXO metrics indicator
    utxo_signal = 'BUY' if get_utxo_metrics(ticker_df)['vortex_pos'].iloc[-1] <=\
                    UTXO_MULTIPLIER * close_prices.iloc[-1] else 'SELL'
    utxo_weight = weights['UTXO Metrics']

    # BTC Energy Value Oscillator indicator
    btc_ev_oscillator = get_btc_energy_value_oscillator(ticker_df)
    if btc_ev_oscillator < BTC_EV_OSCILLATOR_BUY_THRESHOLD:
        btc_ev_oscillator_signal = 'BUY'
    else:
        btc_ev_oscillator_signal = 'SELL'
    btc_ev_oscillator_weight = weights['BTC Energy Value Oscillator']

    # Metcalfe's Law indicator
    metcalfe_ratio = get_metcalfe_law(ticker_df)
    if metcalfe_ratio <= METCALFE_MULTIPLIER * close_prices.iloc[-1]:
        metcalfe_signal = 'BUY'
    else:
        metcalfe_signal = 'SELL'
    metcalfe_weight = weights["Metcalfe's Law"]

    # Puell Multiple indicator
    puell_ratio = get_puell_multiple(ticker_df)
    if puell_ratio < threshold:
        puell_signal = 'BUY'
    else:
        puell_signal = 'SELL'
    puell_weight = weights['Puell Multiple']

    # Calculate overall score and recommendation
    score = (rsi_weight * rsi) + (macd_weight * macd[0]) + (bb_weight * ((close_prices - bb_lower) / (bb_upper - bb_lower))) + \
            (hash_ribbon_weight * (hash_ribbon_signal == 'BUY')) + (volume_weight * (volume_signal == 'BUY')) + \
            (utxo_weight * (utxo_signal == 'BUY')) + (btc_ev_oscillator_weight * (btc_ev_oscillator_signal == 'BUY')) + \
            (metcalfe_weight * (metcalfe_signal == 'BUY')) + (puell_weight * (puell_signal == 'BUY'))

    if score.values[0] >= 5:
        return 'STRONG BUY'
    elif score.values[0] >= 3:
        return 'BUY'
    elif score.values[0] <= -5:
        return 'STRONG SELL'
    elif score.values[0] <= -3:
        return 'SELL'
    else:
        return 'HOLD'

In [26]:
def optimize_parameters(df):
    def objective_function(params, *args):
        fast_period, slow_period = int(params[0]), int(params[1])
        ticker_data = args[0]
        trade_rec = get_trade_recommendation(df, fast_period, slow_period, weights, threshold)

        # Asumiendo una estrategia de retorno simple (puede ser modificada según necesidades)
        returns = (ticker_data['close'].pct_change() * (trade_rec == 'BUY')).cumsum()
        return -returns.iloc[-1]  # Maximizar el retorno

    # Buscar los parámetros óptimos usando la función 'brute' de SciPy
    optimization_result = brute(
        objective_function,
        (
            (OPTIMIZATION_PERIODS[0], OPTIMIZATION_PERIODS[1]),  # Rango para el período rápido de MACD
            (OPTIMIZATION_PERIODS[0], OPTIMIZATION_PERIODS[2]),  # Rango para el período lento de MACD
        ),
        args=(df,),
        full_output=True,
        finish=None,  # No realizar optimización adicional
    )

    optimal_fast_period, optimal_slow_period = int(optimization_result[0][0]), int(optimization_result[0][1])
    return optimal_fast_period, optimal_slow_period

In [31]:
fast_period, slow_period = optimize_parameters(df)
# Función principal
def run_bot_for_ticker(ccxt_ticker, trading_ticker):
    ticker_data = fetch_data(ccxt_ticker)
    if ticker_data is not None:
        optimal_fast_period, optimal_slow_period = optimize_parameters(ticker_data)
        trade_rec_type = get_trade_recommendation(ticker_data, fast_period, slow_period, weights, threshold)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}  TRADING RECOMMENDATION: {trade_rec_type}')
        
        # Ejecutar órdenes, gestión de riesgos, stop-loss y take-profit aquí
        # ...
        
        return trade_rec_type
    else:
        print("No se pudo obtener datos del ticker.")
        return None

In [32]:
# Ejecutar el bot
UTXO_MULTIPLIER = 2
run_bot_for_ticker(CCXT_TICKER_NAME, TRADING_TICKER_NAME)

11/04/2023 21:01:28  TRADING RECOMMENDATION: HOLD


'HOLD'

In [ ]:
def get_trade_recommendation(ticker_df, fast_period, slow_period):
    close_prices = ticker_df['close']

    # RSI
    delta = close_prices.diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=RSI_PERIOD).mean()
    avg_loss = loss.rolling(window=RSI_PERIOD).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # MACD
    ema_fast = close_prices.ewm(span=fast_period).mean()
    ema_slow = close_prices.ewm(span=slow_period).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=9).mean()

    # Bollinger Bands
    bb_middle = close_prices.rolling(window=BB_PERIOD).mean()
    bb_std = close_prices.rolling(window=BB_PERIOD).std()
    bb_upper = bb_middle + (BB_STD_DEV * bb_std)
    bb_lower = bb_middle - (BB_STD_DEV * bb_std)

    # Condición de compra
    buy_condition = (
        (rsi.iloc[-1] <= RSI_OVERSOLD) &
        (macd.iloc[-1] > signal_line.iloc[-1]) &
        (close_prices.iloc[-1] <= bb_lower.iloc[-1])
    )

    # Condición de venta
    sell_condition = (
        (rsi.iloc[-1] >= RSI_OVERBOUGHT) &
        (macd.iloc[-1] < signal_line.iloc[-1]) &
        (close_prices.iloc[-1] >= bb_upper.iloc[-1])
    )

    if buy_condition:
        return 'BUY'
    elif sell_condition:
        return 'SELL'
    else:
        return 'WAIT'
    
    # Add Hash Ribbon indicator
    hashrate_30d_ma, hashrate_60d_ma = get_hash_ribbon_data()
    if hashrate_30d_ma is None or hashrate_60d_ma is None:
        print("Error fetching Hash Ribbon data, skipping this indicator")
    else:
        if hashrate_30d_ma > hashrate_60d_ma:
            hash_ribbon_signal = 'BUY'
        else:
            hash_ribbon_signal = 'SELL'
    
    # Add Volume indicator
    volume = calculate_volume(ticker_df)
    volume_signal = 'BUY' if volume.iloc[-1] > volume.iloc[-2] else 'SELL'
    
     # Add UTXO metrics indicator
    utxo = get_utxo()
    utxo_signal = 'BUY' if utxo <= UTXO_MULTIPLIER * close_prices.iloc[-1] else 'SELL'

    # Add BTC Energy Value Oscillator indicator
    btc_ev_oscillator = get_btc_energy_value_oscillator()
    btc_ev_oscillator_signal = 'BUY' if btc_ev_oscillator < BTC_EV_OSCILLATOR_BUY_THRESHOLD else 'SELL'

    # Add Metcalfe's Law indicator
    metcalfe_ratio = get_metcalfe_law(ticker_df)
    metcalfe_signal = 'BUY' if metcalfe_ratio <= METCALFE_MULTIPLIER * close_prices.iloc[-1] else 'SELL'

   
        # Add Puell Multiple indicator
    puell_multiple = get_puell_multiple()
    if puell_multiple is None:
        print("Error fetching Puell Multiple data, skipping this indicator")
    else:
        if puell_multiple > 4:
            puell_multiple_signal = 'SELL'
            puell_multiple_buy = False
        elif puell_multiple < 1:
            puell_multiple_signal = 'BUY'
            puell_multiple_buy = True
        else:
            puell_multiple_signal = 'WAIT'

    # Add BTC energy value oscillator indicator
    energy_value_oscillator = get_btc_energy_value_oscillator()
    if energy_value_oscillator is None:
        print("Error fetching Energy Value Oscillator data, skipping this indicator")
    else:
        if energy_value_oscillator > 3:
            energy_value_oscillator_signal = 'SELL'
        elif energy_value_oscillator < -2:
            energy_value_oscillator_signal = 'BUY'
        else:
            energy_value_oscillator_signal = 'WAIT'

    # Add UTXO metrics indicator
    utxo = get_utxo_metrics()
    if utxo is None:
        print("Error fetching UTXO metrics data, skipping this indicator")
    else:
        if utxo['utxo_1w_change'] < -15 and utxo['utxo_3m_change'] < -15 and utxo['utxo_6m_change'] < -15:
            utxo_signal = 'BUY'
            utxo_buy = True
        elif utxo['utxo_1w_change'] > 30 and utxo['utxo_3m_change'] > 30 and utxo['utxo_6m_change'] > 30:
            utxo_signal = 'SELL'
            utxo_buy = False
        else:
            utxo_signal = 'WAIT'

    # Combine all signals
    if rsi_signal == 'BUY' and macd_signal == 'BUY' and hash_ribbon_signal == 'BUY'\
                        and volume_signal == 'BUY' and puell_multiple_signal == 'BUY'\
                        and energy_value_oscillator_signal == 'BUY' and utxo_signal == 'BUY':
        final_signal = 'BUY'
    elif rsi_signal == 'SELL' and macd_signal == 'SELL' and hash_ribbon_signal == 'SELL'\
                        and volume_signal == 'SELL' and puell_multiple_signal == 'SELL' and energy_value_oscillator_signal == 'SELL' and utxo_signal == 'SELL':
        final_signal = 'SELL'
    else:
        final_signal = 'WAIT'

    return final_signal